This document parses and transforms CSV High Frontier 4 patent cards into JSON.

# Preamble

## CSV import

In [1]:
import csv
import glob
from os.path import splitext, basename
import unittest

def read_cards(deckname, fields):
    with open(f'decks/{deckname}.csv', 'r') as fh:
        reader = csv.DictReader(fh, fieldnames=fields)
        next(reader) # Drop row 1, which contains multi-collumn cells describing groups of fields (e.g. Thruster and Support Requirements)
        next(reader) # Drop row 2, which contains field names
        deck = list(reader)
        
    for i, card in enumerate(deck):
        split_keys(card)
               
    return deck    

def side_from_index(i):
    if i % 2 == 0:
        return 'Front'
    return 'Reverse'

def split_keys(card):
    namespaced_keys = [k for k in card if '.' in k]
    for key in namespaced_keys:
        k0, k1 = key.split('.')
        if not card.get(k0):
            card[k0] = {}
        card[k0][k1] = card.pop(key)

def chunklist(l, n):
    return zip(*[iter(l)] * 2)
               
class TestUtils(unittest.TestCase):
    def test_split(self):
        card = {
            'mass': 0,
            'radHard': 4,
            'thrustTriangle.thrust': 3,
            'thrustTriangle.fuelConsumption': '2',
            'supportRequirements.⟛ Generator': '1'
        }
        
        expected_result = {
            'mass': 0,
            'radHard': 4,
            'thrustTriangle': {
                'thrust': 3,
                'fuelConsumption': '2',
            },
            'supportRequirements': {'⟛ Generator': '1'},
        }
        
        split_keys(card)
        self.assertEqual(card, expected_result)
        
unittest.main(argv=['', 'TestUtils'], verbosity=2, exit=False)

test_split (__main__.TestUtils) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## Debug tools

In [2]:
import random

def draw(deck):
    return random.choice(deck)

## Common schema

These schemas are "leaves" in the sense that they don't refer to any other schemas.

In [3]:
from jsonschema import validate, ValidationError
import json
from os import path, makedirs

SCHEMA_LOCATION = 'https://raw.githubusercontent.com/ouroboros8/high-frontier-cards/main/schema/'

def save_schema(schema):
    filepath = path.join('schema/', schema['$id'].removeprefix(SCHEMA_LOCATION))
    makedirs(path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w') as f:
        json.dump(schema, f, ensure_ascii=False, indent=2, sort_keys=True)
    
    return schema

thrust_triangle_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'thrust-triangle.schema.json'),
        'title': 'Thrust Triangle',
        'description': 'A thrust triangle on a card.',
        'type': 'object',
        'properties': {
            'thrust': {
                'description': 'The base thrust of the thrust triangle',
                'type': 'integer',
            },
            'fuelConsumption': {
                'description': 'A string representation of the base fuel consumption as an integer or fraction.',
                'type': 'string',
            },
            'fuelType': {
                'description': 'The type of fuel the thrust triangle consumes.',
                'enum': ['water', 'dirt', 'isotope'],
            },
            'afterburn': {
                'description': 'The number of afterburns the thruster can perform.',
                'type': 'integer',
            },
            'push': {
                'description': 'Whether the thrust triangle is pushable.',
                'type': 'boolean',
            },
            'solar': {
                'description': 'Whether the thrust triangle is solar.',
                'type': 'boolean',
            }
        },
        'required': ['thrust', 'fuelConsumption', 'fuelType'],
        'additionalProperties': False,
    }
)

isru_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'isru.schema.json'),
        'title': 'ISRU',
        'description': "The card's ISRU rating and platform icons.",
        'type': 'object',
        'properties': {
            'rating': {
                'description': 'The ISRU rating.',
                'type': 'integer',
            },
            'platforms': {
                'description': 'An array of platforms icons on the card.',
                'type': 'array',
                'items': {
                    'enum': ['raygun', 'buggy', 'missile'],                
                },
            },
        },
        'required': ['rating', 'platforms'],
        'additionalProperties': False,
    }
)

generator_support_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'supports/generator.schema.json'),
        'title': 'Generator Support',
        'properties': {
            'type': { 'const': 'generator' },
            'subtypes': {
                'type': 'array',
                'items': { 'enum': ['e', '⟛'] }
            },
        },
        'required': ['type', 'subtypes'],
        'additionalProperties': False,
    }
)

reactor_support_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'supports/reactor.schema.json'),
        'title': 'Reactor Support',
        'properties': {
            'type': { 'const': 'reactor' },
            'subtypes': {
                'type': 'array',
                'items': { 'enum': ['X', '∿', '💣'] }
            },
        },
        'required': ['type', 'subtypes'],
        'additionalProperties': False,
    }
)

cooling_support_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'supports/cooling.schema.json'),
        'title': 'Cooling Support Requirement',
        'properties': {
            'type': { 'const': 'cooling' },
            'therms': { 'type': 'integer' },
        },
        'required': ['type', 'therms'],
        'additionalProperties': False,
    }
)

support_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'supports/any.schema.json'),
        'title': 'Support',
        'oneOf': [
            { '$ref': generator_support_schema['$id'] },
            { '$ref': reactor_support_schema['$id'] },
            { '$ref': cooling_support_schema['$id'] },
        ]
    }
)

from jsonschema import validate, ValidationError
def validate_deck(deck, schema):
    for card in deck:
        name = '%s/%s' % (card['front']['name'], card['back']['name'])
        try:
            print('Validating %s: %s' % (card['type'], name))
            validate(card, schema)
        except ValidationError as e:
            print('Error validating %s: %s' % (card['type'], name))
            print(json.dumps(card, ensure_ascii=False, indent=4))
            raise e
    print('ok')

## Common transformations

In [4]:
import unittest

def supports(card):
    supports = []
    
    # Special case for Colliding FRC 3He-D Fusion TeraWatt Thruster
    if card.get('supportRequirements', {}).get('Any Reactor', False):
        any_reactor = {'type': 'reactor', 'subtypes': ['∿', '💣', 'X']}
        supports += [any_reactor, any_reactor]      

    reactor_subtypes = []    
    # Other than the Colliding FRC 3He-D Fusion TeraWatt Thruster, cards only need 1 reactor symbol. If we have multiple symbols, that means that we need any one of them, not several.
    for t in ['∿', '💣', 'X']:
        if card.get('supportRequirements', {}).get(f'{t} Reactor', False):
            reactor_subtypes.append(t)

    if len(reactor_subtypes) != 0:
        supports.append(
            {
                'type': 'reactor',
                'subtypes': reactor_subtypes
            }
        )

    # Cards only ever need exactly one generator type
    if card.get('supportRequirements', {}).get('e Generator', False):
        supports.append(
            {
                'type': 'generator',
                'subtypes': ['e']
            }
        )
    elif card.get('supportRequirements', {}).get('⟛ Generator', False):
        supports.append(
            {
                'type': 'generator',
                'subtypes': ['⟛']
            }
        )

    therms = card.get('supportRequirements', {}).get('Cooling', 0)
    if therms != 0:
        supports.append({'type': 'cooling', 'therms': therms})
    
    return supports

class SupportTests(unittest.TestCase):
    
    def test_no_supports(self):
        card = {}
        self.assertEqual(supports(card), [])
        
    def test_single_generator(self):
        card = {'supportRequirements': {'⟛ Generator': True}}
        self.assertEqual(supports(card), [{'type': 'generator', 'subtypes': ['⟛']}])

    def test_single_reactor(self):
        card = {'supportRequirements': {'💣 Reactor': True}}
        self.assertEqual(supports(card), [{'type': 'reactor', 'subtypes': ['💣']}])
        
    def test_multiple_reactors(self):
        card = {'supportRequirements': {'💣 Reactor': True, '∿ Reactor': True, 'X Reactor': True}}
        self.assertEqual(supports(card), [{
            'type': 'reactor',
            'subtypes': ['∿', '💣', 'X']
        }])

    def test_reactor_and_generator(self):
        card = {'supportRequirements': {'💣 Reactor': True, 'X Reactor': False, 'e Generator': True}}
        self.assertEqual(supports(card), [{'type': 'reactor', 'subtypes': ['💣']}, {'type': 'generator', 'subtypes': ['e']}])
        
    def test_therms(self):
        card = {'supportRequirements': {'Cooling': 2}}
        self.assertEqual(supports(card), [{'type': 'cooling', 'therms': 2}])
    
    def test_colliding_frc_special_case(self):
        '''
        The Colliding FRC 3He-D Fusion GW Thruster has the 'Any' reactor subtype in the spreadsheet. It actually requires TWO of any reactor. We handle this as a special case which requires two Reactor type Support Requirements.
        '''
        card = {'supportRequirements': {'Any Reactor': True}}
        any_reactor = {'type': 'reactor', 'subtypes': ['∿', '💣', 'X']}
        self.assertEqual(supports(card), [any_reactor, any_reactor])
        

unittest.main(argv=['', 'SupportTests'], verbosity=2, exit=False)

test_colliding_frc_special_case (__main__.SupportTests)
The Colliding FRC 3He-D Fusion GW Thruster has the 'Any' reactor subtype in the spreadsheet. It actually requires TWO of any reactor. We handle this as a special case which requires two Reactor type Support Requirements. ... ok
test_multiple_reactors (__main__.SupportTests) ... ok
test_no_supports (__main__.SupportTests) ... ok
test_reactor_and_generator (__main__.SupportTests) ... ok
test_single_generator (__main__.SupportTests) ... ok
test_single_reactor (__main__.SupportTests) ... ok
test_therms (__main__.SupportTests) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.005s

OK


In [5]:
import unittest

def get_isru(robonaut):
    return {
        'rating': robonaut['ISRU']['ISRU'],
        'platforms': [p.lower() for p in ['Raygun', 'Buggy', 'Missile'] if robonaut.get('ISRU').get(p)],
    }

class ISRUTests(unittest.TestCase):
    
    def test_raygun(self):
        card = {'ISRU': {'ISRU': 2, 'Raygun': True, 'Buggy': False, 'Missile': False}}
        self.assertEqual(get_isru(card), {'rating': 2, 'platforms': ['raygun']})
    
    def test_buggy(self):
        card = {'ISRU': {'ISRU': 0, 'Raygun': False, 'Buggy': True, 'Missile': False}}
        self.assertEqual(get_isru(card), {'rating': 0, 'platforms': ['buggy']})
        
    def test_missile(self):
        card = {'ISRU': {'ISRU': 1, 'Raygun': False, 'Buggy': False, 'Missile': True}}
        self.assertEqual(get_isru(card), {'rating': 1, 'platforms': ['missile']})

    def test_multiple(self):
        card = {'ISRU': {'ISRU': 1, 'Raygun': True, 'Buggy': False, 'Missile': True}}
        self.assertEqual(get_isru(card), {'rating': 1, 'platforms': ['raygun', 'missile']})

unittest.main(argv=['', 'ISRUTests'], verbosity=2, exit=False)

test_buggy (__main__.ISRUTests) ... ok
test_missile (__main__.ISRUTests) ... ok
test_multiple (__main__.ISRUTests) ... ok
test_raygun (__main__.ISRUTests) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK


In [6]:
def str_to_int(s):
    if s == '':
        return 0
    else:
        return int(s)

def str_to_bool(s):
    if s == '0' or s == '':
        return False
    else:
        return True

def intify(card, fields):
    for k in fields:
        if '.' in k:
            k0, k1 = k.split('.')
            card[k0][k1] = str_to_int(card[k0][k1])
        else:
            card[k] = str_to_int(card[k])
            
def boolify(card, fields):
    for k in fields:
        if '.' in k:
            k0, k1 = k.split('.')
            card[k0][k1] = str_to_bool(card[k0][k1])
        else:
            card[k] = str_to_bool(card[k])
            
def lowercase(card, fields):
    for k in fields:
        if '.' in k:
            k0, k1 = k.split('.')
            card[k0][k1] = card[k0][k1].lower()
        else:
            card[k] = card[k].lower()
            
def deckname_to_type(deckname):
    return {
        'Bernals': 'bernal',
        'Colonists': 'colonist',
        'Freighters': 'freighter',
        'Generators': 'generator',
        'GW Thrusters': 'gw/tw thrusters',
        'Radiators': 'radiator',
        'Reactors': 'reactor',
        'Refineries': 'refinery',
        'Robonauts': 'robonaut',
        'Thrusters': 'thruster',
    }[deckname]

def has_empty_thrust_triangle(card):
    thrust_triangle = card.get('thrustTriangle', None)
    if thrust_triangle is not None:
        if thrust_triangle.get('thrust', '') == '':
            return True
    return False

def build_deck(deckname, fields, int_fields, bool_fields, uppercased_fields):
    deck = []
    for front, back in chunklist(read_cards(deckname, fields), 2):
        spectral_type = front.get('spectralType')
        
        for side in [front, back]:
            
            omit_fields = []
            if has_empty_thrust_triangle(side):
                side.pop('thrustTriangle')
                omit_fields += filter(lambda x: x.startswith('thrustTriangle.'), fields)
                
            intify(side, [f for f in int_fields if f not in omit_fields])
            boolify(side, [f for f in bool_fields if f not in omit_fields])
            lowercase(side, [f for f in uppercased_fields if f not in omit_fields])
            side.pop('spectralType')
            side['supportRequirements'] = supports(side)
            
            isru = side.get('ISRU', None)
            if isru:
                side['ISRU'] = get_isru(side)

        deck.append({
            'cardId': 0, # placeholder
            'type': deckname_to_type(deckname),
            'spectralType': spectral_type,
            'front': front,
            'back': back,
        })
    return deck

# Thrusters

## Import

In [7]:
'''
Thrusters
'''

fields = [
    'name',
    'spectralType',
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'thrustTriangle.fuelConsumption',
    'thrustTriangle.fuelType',
    'bonusPivots',
    'thrustTriangle.afterburn',
    'thrustTriangle.push',
    'thrustTriangle.solar',
    'supportRequirements.⟛ Generator',
    'supportRequirements.e Generator',
    'supportRequirements.X Reactor',
    'supportRequirements.∿ Reactor',
    'supportRequirements.💣 Reactor',
    'supportRequirements.Cooling',
    'ability',
]

int_fields = [
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'thrustTriangle.afterburn',
    'supportRequirements.Cooling',
    'bonusPivots',
]

bool_fields = [
    'thrustTriangle.push',
    'thrustTriangle.solar',
    'supportRequirements.⟛ Generator',
    'supportRequirements.e Generator',
    'supportRequirements.X Reactor',
    'supportRequirements.∿ Reactor',
    'supportRequirements.💣 Reactor',
]

uppercased_fields = [
    'thrustTriangle.fuelType',
]

thrusters = build_deck('Thrusters', fields, int_fields, bool_fields, uppercased_fields)

## Validation

In [ ]:
import json
from os import path

thruster_white_side = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        'title': 'Thruster White-Side',
        '$id': path.join(SCHEMA_LOCATION, 'thruster/white-side.schema.json'),
        'type': 'object',
        'properties': {
            'name': { 'type': 'string' },
            'mass': { 'type': 'integer' },
            'radHard': { 'type': 'integer' },
            'thrustTriangle': { '$ref': thrust_triangle_schema['$id'] },
            'supportRequirements': {
                'type': 'array',
                'items': { '$ref': support_schema['$id'] }
            },
            'bonusPivots' : { 'type': 'integer' },
            'ability': { 'type': 'string' },
        },
        'required': ['name', 'mass', 'radHard', 'thrustTriangle', 'supportRequirements'],
        'additionalProperties': False,
    }
)

thruster_black_side = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        'title': 'Thruster Black-Side',
        '$id': path.join(SCHEMA_LOCATION, 'thruster/black-side.schema.json'),
        'type': 'object',
        'properties': {
            'name': { 'type': 'string' },
            'mass': { 'type': 'integer' },
            'radHard': { 'type': 'integer' },
            'thrustTriangle': { '$ref': thrust_triangle_schema['$id'] },
            'supportRequirements': {
                'type': 'array',
                'items': { '$ref': support_schema['$id'] }
            },
            'bonusPivots' : { 'type': 'integer' },
            'ability': { 'type': 'string' },
        },
        'required': ['name', 'mass', 'radHard', 'thrustTriangle', 'supportRequirements'],
        'additionalProperties': False,
    }
)

thruster_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        'title': 'Thruster',
        '$id': path.join(SCHEMA_LOCATION, 'thruster/patent.schema.json'),
        'type': 'object',
        'properties': {
            'cardId': { 'type': 'integer' },
            'type': { 'const': 'thruster' },
            'spectralType': {
                'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
            },
            'front': { '$ref': thruster_white_side['$id'] },
            'back': { '$ref': thruster_black_side['$id'] }
        },
        'required': ['cardId', 'type', 'spectralType', 'front', 'back'],
        'additionalProperties': False,
    }
)

validate_deck(thrusters, thruster_schema)

Validating thruster: Ablative Plate/Ablative Nozzle
Validating thruster: De Laval Nozzle/Magnetic Nozzle
Validating thruster: Dumbo/Timberwind
Validating thruster: Hall Effect/Ion Drive
Validating thruster: Mass Driver/MPD T-wave
Validating thruster: Metastable Helium/n-6Li Microfission
Validating thruster: Monoatomic Plug Nozzle/Vortex Confined Nozzle
Validating thruster: Photon Heliogyro/Electric Sail
Validating thruster: Photon Kite Sail/Mag Sail
Validating thruster: Ponderomotive VASIMR/Pulsed Plasmoid
Validating thruster: Pulsed Inductive/Dual-Stage 4-Grid
Validating thruster: Re Solar Moth/Colliding Beam H-B Fusion
ok


# Robonauts

In [ ]:
'''
Robonauts
'''

fields = [
    'name',
    'spectralType',
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'thrustTriangle.fuelConsumption',
    'thrustTriangle.fuelType',
    'thrustTriangle.afterburn',
    'thrustTriangle.push',
    'thrustTriangle.solar',
    'ISRU.ISRU',
    'ISRU.Missile',
    'ISRU.Raygun',
    'ISRU.Buggy',
    'supportRequirements.⟛ Generator',
    'supportRequirements.e Generator',
    'supportRequirements.X Reactor',
    'supportRequirements.∿ Reactor',
    'supportRequirements.💣 Reactor',
    'supportRequirements.Cooling',
    'ability',
]

int_fields = [
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'thrustTriangle.afterburn',
    'supportRequirements.Cooling',
    'ISRU.ISRU',
]

bool_fields = [
    'thrustTriangle.push',
    'thrustTriangle.solar',
    'supportRequirements.⟛ Generator',
    'supportRequirements.e Generator',
    'supportRequirements.X Reactor',
    'supportRequirements.∿ Reactor',
    'supportRequirements.💣 Reactor',
    'ISRU.Missile',
    'ISRU.Buggy',
    'ISRU.Raygun',
]

uppercased_fields = [
    'thrustTriangle.fuelType',
]

robonauts = build_deck('Robonauts', fields, int_fields, bool_fields, uppercased_fields)

draw(robonauts)

{'cardId': 0,
 'type': 'robonaut',
 'spectralType': 'D',
 'front': {'name': 'Flywheel Tractor',
  'mass': 0,
  'radHard': 5,
  'ability': '',
  'ISRU': {'rating': 3, 'platforms': ['buggy']},
  'supportRequirements': [{'type': 'generator', 'subtypes': ['e']}]},
 'back': {'name': 'Electrophoretic Sandworm',
  'mass': 1,
  'radHard': 5,
  'ability': '',
  'ISRU': {'rating': 1, 'platforms': ['buggy']},
  'supportRequirements': []}}

## Validation

In [ ]:
from jsonschema import validate, ValidationError
from os import path
import json

robonaut_white_side = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'robonaut/white-side.schema.json'),
        'title': 'Robonaut White-Side',
        'type': 'object',
        'properties': {
            'name': { 'type': 'string' },
            'mass': { 'type': 'integer' },
            'radHard': { 'type': 'integer' },
            'thrustTriangle': { '$ref': thrust_triangle_schema['$id'] },
            'supportRequirements': {
                'type': 'array',
                'items': { '$ref': support_schema['$id'] }
            },
            'ISRU' : { '$ref': isru_schema['$id'] },
            'ability': { 'type': 'string' },
        },
        'required': ['name', 'mass', 'radHard', 'ISRU', 'supportRequirements'],
        'additionalProperties': False,
    }
)

robonaut_black_side = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'robonaut/black-side.schema.json'),
        'title': 'Robonaut Black-Side',
        'type': 'object',
        'properties': {
            'name': { 'type': 'string' },
            'mass': { 'type': 'integer' },
            'radHard': { 'type': 'integer' },
            'thrustTriangle': { '$ref': thrust_triangle_schema['$id'] },
            'supportRequirements': {
                'type': 'array',
                'items': { '$ref': support_schema['$id'] }
            },
            'ISRU' : { '$ref': isru_schema['$id'] },
            'ability': { 'type': 'string' },
        },
        'required': ['name', 'mass', 'radHard', 'ISRU', 'supportRequirements'],
        'additionalProperties': False,
    }
)

robonaut_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'robonaut/patent.schema.json'),
        'title': 'Robonaut',
        'type': 'object',
        'properties': {
            'cardId': { 'type': 'integer' },
            'type': { 'const': 'robonaut' },
            'spectralType': {
                'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
            },
            'front': { '$ref': robonaut_white_side['$id'] },
            'back': { '$ref': robonaut_black_side['$id'] },
        },
        'required': ['cardId', 'type', 'spectralType', 'front', 'back'],
        'additionalProperties': False,
    }
)

validate_deck(robonauts, robonaut_schema)

Validating robonaut: Blackbody-Pumped Laser/Fissile Aerosol Laser
Validating robonaut: Cat Fusion Z-pinch Torch/H-B Cat Inertial
Validating robonaut: Flywheel Tractor/Electrophoretic Sandworm
Validating robonaut: Free Electron Laser/Wakefield e-Beam
Validating robonaut: Kuck Mosquito/Ablative Laser
Validating robonaut: MET Steamer/Nanobot
Validating robonaut: Neutral Beam/D-D Fusion Inertial
Validating robonaut: Nuclear Drill/Helical Railgun
Validating robonaut: Phase-Locked Diode Laser/Lorentz-Propelled Microprobe
Validating robonaut: Rock Splitter/MagBeam
Validating robonaut: Solar-Pumped MHD Exciplex Laser/Quantum Cascade Laser
Validating robonaut: Tungsten Resistojet/MITEE Arcjet
ok


# Refineries

In [20]:
'''
Refineries
'''

fields = [
    'Name',
    'Spectral Type',
    'Mass',
    'Rad-Hard',
    'Air Eater',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
]

bool_fields = [
    'Air Eater',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
]

refineries = read_cards('Refineries', fields)
for i, card in enumerate(refineries):
    if i % 2 == 0:
        spectral_type = card.get('Spectral Type')
    card['Spectral Type'] = spectral_type
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Support Requirements'] = supports(card)

draw(refineries)

{'Name': 'Femtochemistry',
 'Spectral Type': 'D',
 'Mass': 2,
 'Rad-Hard': 8,
 'Air Eater': False,
 'Ability': 'SCAVENGING: If Colocated, doubles FTs during site refuel.',
 'Support Requirements': []}

# Reactors

In [ ]:
'''
Reactors
'''

fields = [
    'Name',
    'Spectral Type',
    'Mass',
    'Rad-Hard',
    'Subtypes.X',
    'Subtypes.∿',
    'Subtypes.💣',
    'Movement Modifiers.Thrust',
    'Movement Modifiers.Fuel Consumption',
    'Air Eater',
    'Support Requirements.⟛ Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.Cooling',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Movement Modifiers.Thrust',
    'Support Requirements.Cooling',
]

bool_fields = [
    'Subtypes.X',
    'Subtypes.∿',
    'Subtypes.💣',
    'Air Eater',
    'Support Requirements.⟛ Generator',
    'Support Requirements.X Reactor',
]

def has_movement_modifiers(card):
    return card.get('Movement Modifiers', {}).get('Thrust', '') != ''

reactors = read_cards('Reactors', fields)
for i, card in enumerate(reactors):
    if i % 2 == 0:
        spectral_type = card.get('Spectral Type')
    card['Spectral Type'] = spectral_type
    if has_movement_modifiers(card):
        intify(card, int_fields)
        boolify(card, bool_fields)
    else:
        card.pop('Movement Modifiers')
        intify(card, filter(lambda f: not f.startswith('Movement Modifiers.'), int_fields))
        boolify(card, filter(lambda f: not f.startswith('Movement Modifiers.'), bool_fields))
    card['Support Requirements'] = supports(card)
    card['Subtypes'] = [k for k in card['Subtypes'].keys() if card['Subtypes'][k]]

draw(reactors)

{'Name': 'D-T Fusion Tokamak',
 'Spectral Type': 'H',
 'Mass': 3,
 'Rad-Hard': 5,
 'Air Eater': False,
 'Ability': '',
 'Subtypes': ['∿'],
 'Movement Modifiers': {'Thrust': 3, 'Fuel Consumption': '1/2'},
 'Support Requirements': []}

# Radiators

In [ ]:
'''
Radiators
'''

fields = [
    'Name',
    'Spectral Type',
    'Light.Mass',
    'Light.Rad-Hard',
    'Light.Therms',
    'Heavy.Mass',
    'Heavy.Rad-Hard',
    'Heavy.Therms',
    'Support Requirements.e Generator',
    'Ability',
]

int_fields = [
    'Light.Mass',
    'Light.Rad-Hard',
    'Light.Therms',
    'Heavy.Mass',
    'Heavy.Rad-Hard',
    'Heavy.Therms',
]

bool_fields = [
    'Support Requirements.e Generator',
]

radiators = read_cards('Radiators', fields)
for i, card in enumerate(radiators):
    if i % 2 == 0:
        spectral_type = card.get('Spectral Type')
    card['Spectral Type'] = spectral_type
    intify(card, int_fields)
    boolify(card, bool_fields)
    # The spreadsheet has a single support requirements field but technically the light and heavy side requirements are distinct, so we duplicate it
    support_reqs = supports(card)
    card.pop('Support Requirements')
    card['Light']['Support Requirements'] = support_reqs
    card['Heavy']['Support Requirements'] = support_reqs

draw(radiators)

{'Name': 'Hula-Hoop',
 'Spectral Type': 'M',
 'Ability': '',
 'Light': {'Mass': 1, 'Rad-Hard': 6, 'Therms': 2, 'Support Requirements': []},
 'Heavy': {'Mass': 2, 'Rad-Hard': 6, 'Therms': 3, 'Support Requirements': []}}

# Generators

In [ ]:
'''
Generators
'''

fields = [
    'Name',
    'Spectral Type',
    'Mass',
    'Rad-Hard',
    'Subtypes.⟛',
    'Subtypes.e',
    'Movement Modifiers.Thrust',
    'Movement Modifiers.Fuel Consumption',
    'Air Eater',
    'Movement Modifiers.Solar',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
    'Support Requirements.Cooling',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Movement Modifiers.Thrust',
    'Support Requirements.Cooling',
]

bool_fields = [
    'Subtypes.⟛',
    'Subtypes.e',
    'Air Eater',
    'Movement Modifiers.Solar',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
]

generators = read_cards('Generators', fields)
for i, card in enumerate(generators):
    if i % 2 == 0:
        spectral_type = card.get('Spectral Type')
    card['Spectral Type'] = spectral_type
    if has_movement_modifiers(card):
        intify(card, int_fields)
        boolify(card, bool_fields)
    else:
        card.pop('Movement Modifiers')
        intify(card, filter(lambda f: not f.startswith('Movement Modifiers.'), int_fields))
        boolify(card, filter(lambda f: not f.startswith('Movement Modifiers.'), bool_fields))
    card['Support Requirements'] = supports(card)
    card['Subtypes'] = [k for k in card['Subtypes'].keys() if card['Subtypes'][k]]
    
draw(generators)

{'Name': 'AMTEC Thermoelectric',
 'Spectral Type': 'C',
 'Mass': 1,
 'Rad-Hard': 6,
 'Air Eater': True,
 'Ability': '',
 'Subtypes': ['⟛', 'e'],
 'Support Requirements': []}

# GW Thrusters

In [ ]:
'''
GW Thrusters
'''

fields = [
    'Name',
    'Type',
    'Spectral Type',
    'Promotion Colony',
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Fuel Consumption',
    'Thrust Triangle.Afterburn',
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.Any Reactor',
    'Support Requirements.Cooling',
    'Future',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Support Requirements.Cooling',
    'Thrust Triangle.Afterburn',
]

bool_fields = [
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.Any Reactor',
]

gwthrusters = read_cards('GW Thrusters', fields)
for i, card in enumerate(gwthrusters):
    if i % 2 == 0:
        card.pop('Future')
        spectral_type = card.get('Spectral Type')
        promotion_colony = card.get('Promotion Colony')
    card['Spectral Type'] = spectral_type
    card['Promotion Colony'] = promotion_colony
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Support Requirements'] = supports(card)
    card['Thrust Triangle']['Fuel Type'] = 'Isotope'
    card['Deck'] = 'GW/TW Thrusters' # Workaround for / in deck name

draw(gwthrusters)

{'Name': 'Solem Medusa Tugged Orion',
 'Type': 'TW Thruster',
 'Spectral Type': 'M',
 'Promotion Colony': 'D',
 'Mass': 6,
 'Rad-Hard': 9,
 'Future': 'LITHIATED AMMONIA ICE STARSHIP FUTURE: Req = Ad astra exit with 10 isotope fuel. Effect = 14 VP.',
 'Thrust Triangle': {'Thrust': 9,
  'Fuel Consumption': '0',
  'Afterburn': 3,
  'Fuel Type': 'Isotope'},
 'Support Requirements': [],
 'Deck': 'GW/TW Thrusters'}

# Freighters

In [ ]:
'''
Freighters
'''

fields = [
    'Name',
    'Type',
    'Spectral Type',
    'Promotion Colony',
    'Mass',
    'Rad-Hard',
    'Load-Limit',
    'Factory Loading Only',
    'Bonus Pivots',
    'Supports Provided.⟛ Generator',
    'Supports Provided.e Generator',
    'Supports Provided.X Reactor',
    'Supports Provided.∿ Reactor',
    'Ability',
    'Future',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Load-Limit',
    'Bonus Pivots',
]

bool_fields = [
    'Factory Loading Only',
    'Supports Provided.⟛ Generator',
    'Supports Provided.e Generator',
    'Supports Provided.X Reactor',
    'Supports Provided.∿ Reactor',
]

freighters = read_cards('Freighters', fields)
for i, card in enumerate(freighters):
    if i % 2 == 0:
        card.pop('Future')
        spectral_type = card.get('Spectral Type')
        promotion_colony = card.get('Promotion Colony')
    card['Spectral Type'] = spectral_type
    card['Promotion Colony'] = promotion_colony
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Supports Provided'] = [k for k in card['Supports Provided'].keys() if card['Supports Provided'][k]]

draw(freighters)

{'Name': 'Fusion Fragment Sail',
 'Type': 'Freighter',
 'Spectral Type': 'V',
 'Promotion Colony': 'M',
 'Mass': 2,
 'Rad-Hard': 1,
 'Load-Limit': 2,
 'Factory Loading Only': False,
 'Bonus Pivots': 1,
 'Ability': 'Immune to flares & radiation belts.',
 'Supports Provided': []}

# Bernals

In [ ]:
'''
Bernals
'''

fields = [
    'Name',
    'Type',
    'Promotion Colony',
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Fuel Consumption',
    'Thrust Triangle.Push',
    'Support Requirements.e Generator',
    'Support Requirements.Cooling',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Support Requirements.Cooling',
]

bool_fields = [
    'Thrust Triangle.Push',
    'Support Requirements.e Generator',
]

bernals = read_cards('Bernals', fields)
for i, card in enumerate(bernals):
    card.pop('Type')
    if i % 2 == 0:
        promotion_colony = card.get('Promotion Colony')
    card['Promotion Colony'] = promotion_colony
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Support Requirements'] = supports(card)
    card['Thrust Triangle']['Fuel Type'] = 'Dirt'

draw(bernals)

{'Name': 'Collimator Lab',
 'Promotion Colony': 'Push',
 'Mass': 10,
 'Rad-Hard': 9,
 'Ability': 'Gain the Powersat faction privilege. Powersat push includes a Bonus Pivot.',
 'Thrust Triangle': {'Thrust': 3,
  'Fuel Consumption': '3',
  'Push': True,
  'Fuel Type': 'Dirt'},
 'Support Requirements': []}

# Colonists

In [ ]:
from functools import reduce
import unittest

def all_f(*fs):
    return lambda *args, **kwargs: all([f(*args, **kwargs) for f in fs])

class FilterTests(unittest.TestCase):
    
    def test_filtering(self):
        fields = [
            'Name',
            'Type',
            'Thrust Triangle.Thrust',
            'Thrust Triangle.Fuel Consumption',
            'Supports.X Reactor',
            'Supports.∿ Reactor',
            'Supports.💣 Reactor',
            'Ability',
        ]
        
        expected_fields = [
            'Name',
            'Type',
            'Ability',
        ]
        
        filter_functions = [lambda x: not x.startswith('Thrust Triangle.'), lambda x: not x.startswith('Supports.')]
        
        filtered_fields = list(filter(all_f(*filter_functions), fields))
        self.assertEqual(filtered_fields, expected_fields)
    
unittest.main(argv=['', 'FilterTests'], verbosity=2, exit=False)

test_filtering (__main__.FilterTests) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [ ]:
'''
Colonists
'''

from functools import reduce

fields = [
    'Name',
    'Type',
    'Specialty',
    'Spectral Type',
    'Promotion Colony',
    'Ideology',
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Fuel Consumption',
    'Thrust Triangle.Fuel Type',
    'Thrust Triangle.Afterburn',
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'Air Eater',
    'Bonus Pivots',
    'ISRU.ISRU',
    'ISRU.Missile',
    'ISRU.Raygun',
    'ISRU.Buggy',
    'Supports.X Reactor',
    'Supports.∿ Reactor',
    'Supports.💣 Reactor',
    'Ability',
    'Future',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Afterburn',
    'ISRU.ISRU',
    'Bonus Pivots',
]

bool_fields = [
    'ISRU.Missile',
    'ISRU.Raygun',
    'ISRU.Buggy',
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'Air Eater',
    'Supports.X Reactor',
    'Supports.∿ Reactor',
    'Supports.💣 Reactor',
]

def has_isru(card):
    return card.get('ISRU', {}).get('ISRU', '') != ''

def na_to_empty(card):
    na_fields = [f for f, v in card.items() if v == 'n/a']
    for f in na_fields:
        card[f] = ''

def get_multirow_values(card):
    return {
        'Type': card['Type'],
        'Specialty': card['Specialty'],
        'Promotion Colony': card['Promotion Colony'],
        'Ideology': card['Ideology']
    }

def set_multirow_values(card, values):
    for k, v in values.items():
        card[k] = v
    
colonists = read_cards('Colonists', fields)
for i, card in enumerate(colonists):
    if i % 2 == 0:
        multirow_values = get_multirow_values(card)
    if i % 2 == 1:
        set_multirow_values(card, multirow_values)
        
    filters = []
    if not has_thruster(card):
        card.pop('Thrust Triangle')
        filters.append(lambda x: not x.startswith('Thrust Triangle.'))
    this_has_isru = has_isru(card)
    if not this_has_isru:
        card.pop('ISRU')
        filters.append(lambda x: not x.startswith('ISRU.'))
    
    na_to_empty(card)
    intify(card, filter(all_f(*filters), int_fields))
    boolify(card, filter(all_f(*filters), bool_fields))
    card['Supports'] = [k for k in card['Supports'].keys() if card['Supports'][k]]
    
    if this_has_isru:
        card['ISRU'] = isru(card)
    
    if card['Type'] == 'Robot':
        card.pop('Ideology')
    elif card['Type'] == 'Human':
        card.pop('Spectral Type')

draw(colonists)

NameError: name 'has_thruster' is not defined

# Validation

## Aggregating card lists

In [ ]:
from itertools import chain

all_cards = list(chain(thrusters, robonauts, refineries, reactors, radiators, generators, gwthrusters, freighters, bernals, colonists))

## Refineries

In [ ]:
from jsonschema import validate, ValidationError
import json

refinery_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Refinery',
    'description': "One of the sides of a patent from the Refinery deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Refineries' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'Air Eater' : { 'type': 'boolean' },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Mass', 'Rad-Hard', 'Support Requirements'],
    'additionalProperties': False,
}

save_schema(refinery_schema)

for card in refineries:
    try:
        print('Validating Refineries:', card['Name'])
        validate(card, refinery_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

## Reactors

In [ ]:
from jsonschema import validate, ValidationError
import json

movement_modifier_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Movement Modifiers',
    'description': 'A movement modifier triangle on a card.',
    'type': 'object',
    'properties': {
        'Thrust': {
            'description': 'The thrust modifier',
            'type': 'integer',
        },
        'Fuel Consumption': {
            'description': 'A string representation of the fuel consumption modifier as an integer or fraction.',
            'type': 'string',
        },
        'Solar': {
            'description': 'Whether the modifier applies solar.',
            'type': 'boolean',
        },
    },
    'required': ['Thrust', 'Fuel Consumption'],
}
save_schema(movement_modifier_schema)

reactor_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Reactor',
    'description': "One of the sides of a patent from the Reactor deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Reactors' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Subtypes': {
            'type': 'array',
            'items': { 'enum': ['X', '∿', '💣'] },
        },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'Movement Modifiers': { '$ref': movement_modifier_schema['$id'] },
        'Air Eater': { 'type': 'boolean' },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Mass', 'Rad-Hard', 'Subtypes', 'Support Requirements'],
    'additionalProperties': False,
}
save_schema(reactor_schema)

for card in reactors:
    try:
        print('Validating Reactor:', card['Name'])
        validate(card, reactor_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

## Radiators

In [ ]:
from jsonschema import validate, ValidationError
import json

radiator_orientation_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Radiator Orientation',
    'description': "One of the orientations of a Radiator card.",
    'type': 'object',
    'properties': {
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Therms': { 'type': 'integer' },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
    },
}
save_schema(radiator_orientation_schema)

radiator_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Radiator',
    'description': "One of the sides of a patent from the Radiator deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Radiators' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Light': { '$ref': radiator_orientation_schema['$id'] },
        'Heavy': { '$ref': radiator_orientation_schema['$id'] },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Light', 'Heavy'],
    'additionalProperties': False,
}
save_schema(radiator_schema)

for card in radiators:
    try:
        print('Validating Radiators:', card['Name'])
        validate(card, radiator_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

## Generators

In [ ]:
from jsonschema import validate, ValidationError
import json

generator_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Generator',
    'description': "One of the sides of a patent from the Generator deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Generators' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Subtypes': {
            'type': 'array',
            'items': { 'enum': ['e', '⟛'] },
        },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'Movement Modifiers': { '$ref': movement_modifier_schema['$id'] },
        'Air Eater': { 'type': 'boolean' },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Mass', 'Rad-Hard', 'Subtypes', 'Support Requirements'],
    'additionalProperties': False,
}
save_schema(generator_schema)

for card in generators:
    try:
        print('Validating Generator:', card['Name'])
        validate(card, generator_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

## GW Thrusters

In [ ]:
from jsonschema import validate, ValidationError
import json

gw_thruster_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'GW Thruster',
    'description': "The GW Thruster side of a card from the GW/TW Thruster deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'GW/TW Thrusters' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Type': { 'const': 'GW Thruster' },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Promotion Colony': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H', 'Push', 'Atmospheric', 'Submarine', 'Astrobiology'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Thrust Triangle': { '$ref': thrust_triangle_schema['$id'] },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Promotion Colony', 'Mass', 'Rad-Hard', 'Thrust Triangle', 'Support Requirements'],
    'additionalProperties': False,
}
save_schema(gw_thruster_schema)

tw_thruster_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'TW Thruster',
    'description': "The TW Thruster side of a card from the GW/TW Thruster deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'GW/TW Thrusters' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Type': { 'const': 'TW Thruster' },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Promotion Colony': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H', 'Push', 'Atmospheric', 'Submarine', 'Astrobiology'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Thrust Triangle': { '$ref': thrust_triangle_schema['$id'] },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'Ability': { 'type': 'string' },
        'Future': {'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Promotion Colony', 'Mass', 'Rad-Hard', 'Thrust Triangle', 'Support Requirements', 'Future'],
    'additionalProperties': False,
}
save_schema(tw_thruster_schema)

gw_tw_thruster_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'GW/TW Thruster',
    'description': "One side of a card from the GW/TW Thruster deck.",
    'oneOf': [
        { '$ref': gw_thruster_schema['$id'] },
        { '$ref': tw_thruster_schema['$id'] },
    ]
}
save_schema(gw_tw_thruster_schema)

for i, card in enumerate(gwthrusters):
    try:
        if i % 2 == 0:
            print('Validating GW Thruster:', card['Name'])
            validate(card, gw_tw_thruster_schema)
        else:
            print('Validating TW Thruster:', card['Name'])
            validate(card, gw_tw_thruster_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

## Freighters

In [ ]:
from jsonschema import validate, ValidationError
import json

basic_freighter_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Basic Freighter',
    'description': "The basic Freighter side of a card from the Freighters patent deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Freighters' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Type': { 'const': 'Freighter' },
        'Spectral Type': {
            'enum': ['Any', 'C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Promotion Colony': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H', 'Push', 'Atmospheric', 'Submarine', 'Astrobiology'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Supports Provided': {
            'type': 'array',
            'item': { 'enum': ['e', '⟛',  'X', '∿', '💣'] },
        },
        'Load-Limit': { 'type': 'integer' },
        'Factory Loading Only': { 'type': 'boolean' },
        'Bonus Pivots': { 'type': 'integer' },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Type', 'Spectral Type', 'Promotion Colony', 'Mass', 'Rad-Hard', 'Load-Limit', 'Supports Provided'],
    'additionalProperties': False,
}
save_schema(basic_freighter_schema)

freighter_fleet_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Freighter Fleet',
    'description': "The Freighter Fleet side of a card from the Freighters patent deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Freighters' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Type': { 'const': 'Freighter Fleet' },
        'Spectral Type': {
            'enum': ['Any', 'C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Promotion Colony': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H', 'Push', 'Atmospheric', 'Submarine', 'Astrobiology'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Supports Provided': {
            'type': 'array',
            'item': { 'enum': ['e', '⟛',  'X', '∿', '💣'] },
        },
        'Load-Limit': { 'type': 'integer' },
        'Factory Loading Only': { 'type': 'boolean' },
        'Bonus Pivots': { 'type': 'integer' },
        'Ability': { 'type': 'string' },
        'Future': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Type', 'Spectral Type', 'Promotion Colony', 'Mass', 'Rad-Hard', 'Load-Limit', 'Supports Provided', 'Future'],
    'additionalProperties': False,
}
save_schema(freighter_fleet_schema)

freighter_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Freighter',
    'description': "One side of a card from the Freighter deck.",
    'oneOf': [
        { '$ref': basic_freighter_schema['$id'] },
        { '$ref': freighter_fleet_schema['$id'] },
    ]
}
save_schema(freighter_schema)

for i, card in enumerate(freighters):
    try:
        print('Validating Freighter:', card['Name'])
        validate(card, freighter_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

## Bernals

In [ ]:
from jsonschema import validate, ValidationError
import json

bernal_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Bernal',
    'description': "The basic Bernal side of a card from the Bernals patent deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Bernals' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Promotion Colony': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H', 'Push', 'Atmospheric', 'Submarine', 'Astrobiology'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Thrust Triangle': { '$ref': thrust_triangle_schema['$id'] },
        'Support Requirements': {
            'type': 'array',
            'item': {
                'oneOf': [
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Promotion Colony', 'Mass', 'Rad-Hard', 'Thrust Triangle', 'Support Requirements'],
    'additionalProperties': False,
}
save_schema(bernal_schema)

for i, card in enumerate(bernals):
    try:
        print('Validating Bernal:', card['Name'])
        validate(card, bernal_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

In [ ]:
from jsonschema import validate

def generate_patent_schemas():
    card_schemas = {
        'Thruster': thruster_schema,
        'Robonaut': robonaut_schema,
        'Refinery': refinery_schema,
        'Reactor': reactor_schema,
        'Radiator': radiator_schema,
        'Generator': generator_schema,
        'GW/TW Truster': gw_tw_thruster_schema,
        'Freighter': freighter_schema,
        'Bernal': bernal_schema,
    }
    
    patent_schemas = []
    
    for cardtype, schema in card_schemas.items():
        if cardtype == 'Refinery':
            plural = 'Refineries'
        else:
            plural = cardtype + 's'

        patent_schemas.append(
            {
                '$schema': 'https://json-schema.org/draft/2020-12/schema',
                'title': f'{cardtype} Patent',
                'description': f'A card from the {plural} patent deck',
                'properties': {
                    'Deck': { 'const': plural },
                    'Front': { '$ref': schema['$id'] },
                    'Reverse': { '$ref': schema['$id'] },
                },
                'required': ['Deck', 'Front', 'Reverse'],
                'additionalProperties': False,
            }
        )
        
    return patent_schemas

def build_deck(clean_cards):
    deckname = clean_cards[0]['Deck']
    return {
        'Name': deckname,
        'Patents': [
            {
                'Deck': deckname,
                'Front': front,
                'Reverse': reverse,
            }
            for front, reverse in chunklist(clean_cards, 2)
        ]
    }

decks = [
    build_deck(cards) for cards in card_sets
]

patent_schemas = generate_patent_schemas()

for schema, deck in zip(patent_schemas, decks):
    try:
        print('Validating', deck['Name'])
        validate(deck, schema)
    except ValidationError as e:
        print(json.dumps(deck, ensure_ascii=False, indent=4))
        raise e

print('ok')

# Output

In [ ]:
from itertools import chain
import json

# card_sets = [
#     thrusters,
#     robonauts,
#     refineries,
#     reactors,
#     radiators,
#     generators,
#     gwthrusters,
#     freighters,
#     bernals,
# ]

# all_cards = list(chain(*card_sets))

print(json.dumps(thrusters, ensure_ascii=False, indent=2))